In [28]:
import pandas as pd
import datetime
import sqlite3
import re
import uuid

In [29]:
# Chemin vers le fichier CSV
csv_file_path = '../data/bronze.csv'

# Charger les données dans un DataFrame
df = pd.read_csv(csv_file_path)

# Afficher les premières lignes du DataFrame pour vérification
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2021,MI,FT,Data Scientist,30400000,CLP,40038,CL,100,CL,L
1,2021,MI,FT,BI Data Analyst,11000000,HUF,36259,HU,50,US,L
2,2020,MI,FT,Data Scientist,11000000,HUF,35735,HU,50,HU,L
3,2021,MI,FT,ML Engineer,8500000,JPY,77364,JP,50,JP,S
4,2022,SE,FT,Lead Machine Learning Engineer,7500000,INR,95386,IN,50,IN,L


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14838 entries, 0 to 14837
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           14838 non-null  int64 
 1   experience_level    14838 non-null  object
 2   employment_type     14838 non-null  object
 3   job_title           14838 non-null  object
 4   salary              14838 non-null  int64 
 5   salary_currency     14838 non-null  object
 6   salary_in_usd       14838 non-null  int64 
 7   employee_residence  14838 non-null  object
 8   remote_ratio        14838 non-null  int64 
 9   company_location    14838 non-null  object
 10  company_size        14838 non-null  object
dtypes: int64(4), object(7)
memory usage: 1.2+ MB


In [31]:
df['work_year'].unique()

array([2021, 2020, 2022, 2023, 2024])

In [32]:
# Supposons que 'df' est votre DataFrame
df.columns = [col.lower().replace(' ', '') for col in df.columns]

# Fonction pour nettoyer les textes
def clean_text(value):
    if isinstance(value, str):
        return value.strip().lower().replace(' ', '')
    return value

# Convertir la colonne 'remote_ratio' en type 'object'
df['remote_ratio'] = df['remote_ratio'].astype(str)

# Nettoyer les valeurs textuelles dans les colonnes
for col in df.select_dtypes(include=['object']):  # sélectionne seulement les colonnes de type objet (texte)
    df[col] = df[col].apply(clean_text)

# Supprimer les doublons
df = df.drop_duplicates()

# Supprimer les lignes avec des valeurs NaN
df = df.dropna()

# Obtenir l'année actuelle
current_year = datetime.datetime.now().year

# Filtrer le DataFrame pour garder les lignes des trois dernières années
df = df[df['work_year'].isin([current_year, current_year-1, current_year-2])]

# Supprimer les colonnes 'salary', 'salary_currency' et 'work_year'
df = df.drop(['salary', 'salary_currency', 'work_year'], axis=1)

df.head()

,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
4,se,ft,leadmachinelearningengineer,95386,in,50,in,l
7,en,ft,datascientist,17684,hu,100,hu,m
8,ex,ft,headofmachinelearning,76309,in,50,in,l
9,en,ft,researchengineer,41809,jp,50,jp,l
10,mi,ft,headofdatascience,60805,in,50,in,l


In [33]:
# Liste des colonnes dans l'ordre souhaité
new_column_order = ['experience_level', 'employment_type', 'job_title', 
                    'employee_residence', 'remote_ratio', 'company_location', 'company_size', 
                    'salary_in_usd']

# Réorganiser les colonnes du DataFrame
df = df[new_column_order]

df.head()

,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,salary_in_usd
4,se,ft,leadmachinelearningengineer,in,50,in,l,95386
7,en,ft,datascientist,hu,100,hu,m,17684
8,ex,ft,headofmachinelearning,in,50,in,l,76309
9,en,ft,researchengineer,jp,50,jp,l,41809
10,mi,ft,headofdatascience,in,50,in,l,60805


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8836 entries, 4 to 14836
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   experience_level    8836 non-null   object
 1   employment_type     8836 non-null   object
 2   job_title           8836 non-null   object
 3   employee_residence  8836 non-null   object
 4   remote_ratio        8836 non-null   object
 5   company_location    8836 non-null   object
 6   company_size        8836 non-null   object
 7   salary_in_usd       8836 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 621.3+ KB


In [35]:
# Chemin du fichier CSV de sortie
output_csv_file = '../data/silver.csv'

# Utiliser to_csv() pour créer le fichier CSV à partir du DataFrame
df.to_csv(output_csv_file, index=False, mode='w')